In [ ]:
-- next_scheduled_run_date 
    -- Histórico da atividade da execução dos jobs (Dirceu Resende)
        -- Nessa visão, podemos identificar quando um job foi iniciado e quando será a próxima execução. A vantagem dessa view sobre a sysjobhistory, é que se o SQL Agent for reiniciado durante a execução de um job, não é gerado registro na sysjobhistory, mas pela sysjobactivity podemos identificar que um job estava em execução e não terminou.

SELECT 
    A.job_id,
    A.[name],
    -- B.session_id,
    B.run_requested_date,
    B.run_requested_source,
    -- B.queued_date,
    B.start_execution_date,
    B.last_executed_step_id,
    B.last_executed_step_date,
    B.stop_execution_date,
    B.job_history_id,
    B.next_scheduled_run_date	
FROM
    msdb.dbo.sysjobs A
    JOIN msdb.dbo.sysjobactivity B ON B.job_id = A.job_id
WHERE NAME IN ('Job_stpLogMonitor')
ORDER BY last_executed_step_date DESC

In [ ]:
-- EXEC msdb.dbo.sp_start_job 'Job_Kill_Session'

EXEC msdb.dbo.sp_start_job 'Job_SimpleCheckDb'

In [ ]:
-- ALTERA PARA "Go to the next step" APÓS NOVO STEP

USE [msdb]
GO
EXEC msdb.dbo.sp_update_jobstep @job_id=N'0c6eb40a-be74-4e36-83c3-0cca5e893463', @step_id=1 , 
		@on_success_action=3
GO

In [ ]:
-- sp_update_schedule

EXEC msdb.dbo.sp_attach_schedule @job_id=N'4b664a7a-17dc-4c22-80dd-36202144b99b',@schedule_id=10
GO
USE [msdb]
GO
EXEC msdb.dbo.sp_update_schedule @schedule_id=10, 
		@freq_type=8, 
		@freq_interval=42, 
		@freq_subday_type=8, 
		@freq_recurrence_factor=1, 
		@active_start_date=20200824, 
		@active_start_time=80000, 
		@active_end_time=225959
GO

In [ ]:
-- sp_update_jobstep ENABLED

EXEC msdb.dbo.sp_update_job @job_name='Job_stpTbLogMonitor_Db_Program_User_Connection', @enabled = 1 -- Disable/Enable

In [ ]:
-- sp_update_jobstep

EXEC msdb.dbo.sp_update_jobstep @job_name=N'Job_LogMonitor', 
        @step_id=2, 
		-- @on_success_action=3, -- Next Job in sucess
	    @step_name = N'step_LogMonitor_CPU',
		@command=N'SELECT 1'